# **Chapter 37: Security Testing Fundamentals**

---

## **Introduction**

Security testing is the process of evaluating a software application to identify vulnerabilities that could be exploited by malicious actors. Unlike functional testing—which verifies that features work as intended—security testing attempts to break the system by thinking like an attacker.

In 2025, the average cost of a data breach reached **$4.88 million** (IBM Security Report), with the average time to identify a breach being **277 days**. Security is not a feature that can be added at the end of development; it must be woven into every phase of the Software Development Life Cycle (SDLC).

This chapter establishes the foundational principles of security testing, the common vulnerabilities defined by OWASP (Open Web Application Security Project), and the methodologies used to identify and mitigate security risks.

---

## **37.1 Security Testing Principles**

### **37.1.1 The CIA Triad**

The foundation of information security rests on three pillars:

**Confidentiality**
- Ensuring that sensitive information is accessible only to authorized individuals
- **Testing Focus**: Encryption, access controls, authentication bypass attempts
- **Example**: Verifying that user A cannot access user B's financial records by manipulating request parameters

**Integrity**
- Maintaining the accuracy and completeness of data throughout its lifecycle
- **Testing Focus**: Data tampering, injection attacks, checksum verification
- **Example**: Ensuring that transaction amounts cannot be modified in transit

**Availability**
- Ensuring that systems and data are accessible when needed by authorized users
- **Testing Focus**: Denial of Service (DoS) vulnerabilities, resource exhaustion
- **Example**: Testing that the application remains responsive under legitimate load and resistant to flooding attacks

### **37.1.2 Defense in Depth**

Security should not rely on a single control point. Defense in depth implements multiple layers of security controls:

```
Defense in Depth Layers:

Layer 7: Application Layer (Input validation, secure coding)
Layer 6: Presentation Layer (HTTPS, secure cookies)
Layer 5: Session Layer (Authentication, authorization)
Layer 4: Transport Layer (TLS 1.3, certificate pinning)
Layer 3: Network Layer (Firewalls, VPNs, segmentation)
Layer 2: Data Layer (Encryption at rest, database security)
Layer 1: Physical Layer (Data center access controls)
```

**Testing Implication**: A tester must verify security at every layer, not just the application boundary.

### **37.1.3 Least Privilege**

Users and processes should operate with the minimum permissions necessary to complete their tasks.

**Testing Approach**:
- Verify that database connections use limited accounts (not `sa` or `root`)
- Test horizontal privilege escalation (accessing other users' data)
- Test vertical privilege escalation (user accessing admin functions)

```python
# VULNERABLE: Application runs with excessive privileges
connection_string = "Server=db;Database=app;User Id=sa;Password=admin123;"

# SECURE: Application uses least privilege account
connection_string = "Server=db;Database=app;User Id=app_readwrite;Password=[encrypted];"
```

### **37.1.4 Fail Secure**

When security controls fail, the system should default to a secure state, not an open one.

**Examples**:
- If authentication service is unavailable, deny access (not grant it)
- If input validation fails, reject the input (not sanitize and accept)
- If encryption keys are corrupted, prevent data access (not bypass encryption)

---

## **37.2 Common Vulnerabilities (OWASP Top 10)**

The OWASP Top 10 represents the most critical security risks to web applications. Every security tester must understand these vulnerabilities intimately.

### **37.2.1 A01: Broken Access Control**

**Description**: Restrictions on what authenticated users are allowed to do are not properly enforced. Attackers can access functionality or data they shouldn't.

**Common Vulnerabilities**:
- **Insecure Direct Object References (IDOR)**: Accessing resources by modifying IDs
- **Path Traversal**: `../../../etc/passwd`
- **Privilege Escalation**: User accessing admin endpoints
- **CORS Misconfiguration**: Allowing malicious sites to make requests

**Testing Example**:

```python
# VULNERABLE CODE
@app.route('/api/orders/<order_id>')
def get_order(order_id):
    # Missing authorization check!
    order = db.query(f"SELECT * FROM orders WHERE id = {order_id}")
    return jsonify(order)

# EXPLOIT
# User A (legitimate) accesses: /api/orders/12345
# User B (attacker) tries: /api/orders/12346 (belongs to User C)
# If no check, attacker sees other users' orders (IDOR)

# SECURE CODE
@app.route('/api/orders/<order_id>')
@login_required
def get_order_secure(order_id):
    # Verify ownership
    order = Order.query.filter_by(id=order_id, user_id=current_user.id).first()
    if not order:
        abort(403)  # Forbidden
    return jsonify(order)
```

**Test Cases**:
1. Access resources with different user IDs in parameters
2. Attempt to access admin endpoints as regular user
3. Use HTTP method tampering (change POST to PUT/DELETE)
4. Test with and without authentication tokens

### **37.2.2 A02: Cryptographic Failures**

**Description**: Failures related to cryptography (formerly "Sensitive Data Exposure"). Either weak algorithms are used, or cryptography is implemented incorrectly.

**Vulnerabilities**:
- Storing passwords in plain text or with weak hashing (MD5, SHA1)
- Using deprecated protocols (SSL, TLS 1.0/1.1)
- Hardcoded encryption keys in source code
- Insufficient key length (RSA < 2048 bits)

**Testing Code**:

```python
# VULNERABLE: Weak hashing
import hashlib
def hash_password(password):
    return hashlib.md5(password.encode()).hexdigest()  # Broken: MD5 is cryptographically weak

# SECURE: Strong hashing with salt
import bcrypt
def hash_password_secure(password):
    salt = bcrypt.gensalt(rounds=12)  # Adaptive cost factor
    return bcrypt.hashpw(password.encode(), salt)

# VULNERABLE: Hardcoded key
ENCRYPTION_KEY = "mysecretkey123"

# SECURE: Key from environment/HSM
import os
from cryptography.fernet import Fernet
ENCRYPTION_KEY = os.environ.get('ENCRYPTION_KEY')
cipher = Fernet(ENCRYPTION_KEY)
```

**Verification Tests**:
- Check password storage algorithms (should be bcrypt, Argon2, or PBKDF2)
- Verify TLS configuration (testssl.sh, SSL Labs scan)
- Search codebase for hardcoded secrets (truffleHog, gitLeaks)
- Verify key rotation policies

### **37.2.3 A03: Injection**

**Description**: Untrusted data is sent to an interpreter as part of a command or query, tricking the interpreter into executing unintended commands.

**Types**:
- **SQL Injection**: `' OR '1'='1`
- **NoSQL Injection**: `{"$gt": ""}`
- **Command Injection**: `; cat /etc/passwd`
- **LDAP Injection**: `*)(uid=*))(&(uid=*`
- **Expression Language Injection**: `${7*7}`

**SQL Injection Testing**:

```python
# VULNERABLE CODE
def search_users(username):
    query = f"SELECT * FROM users WHERE username = '{username}'"
    return db.execute(query)

# ATTACK PAYLOADS
# Input: admin'--
# Result: SELECT * FROM users WHERE username = 'admin'--'
# Comments out password check

# Input: ' OR '1'='1
# Result: SELECT * FROM users WHERE username = '' OR '1'='1'
# Returns all users

# SECURE CODE (Parameterized Queries)
def search_users_secure(username):
    query = "SELECT * FROM users WHERE username = %s"
    return db.execute(query, (username,))  # Parameters escaped automatically

# SECURE CODE (ORM)
def search_users_orm(username):
    return User.query.filter_by(username=username).all()  # ORM handles escaping
```

**Command Injection Testing**:

```python
# VULNERABLE
@app.route('/ping')
def ping_host():
    host = request.args.get('host')
    result = os.system(f"ping -c 1 {host}")  # Dangerous!
    return result

# ATTACK: ?host=google.com; cat /etc/passwd
# Executes: ping -c 1 google.com; cat /etc/passwd

# SECURE
import subprocess
@app.route('/ping')
def ping_secure():
    host = request.args.get('host')
    # Whitelist validation
    if not re.match(r'^[a-zA-Z0-9\-\.]+$', host):
        return "Invalid hostname", 400
    
    # Use list instead of string to prevent shell interpretation
    result = subprocess.run(
        ["ping", "-c", "1", host],
        capture_output=True,
        text=True,
        timeout=5
    )
    return result.stdout
```

### **37.2.4 A04: Insecure Design**

**Description**: Missing or ineffective security controls due to flawed design patterns. This is a broad category covering business logic flaws.

**Examples**:
- **Race Conditions**: Checking balance then withdrawing (time-of-check to time-of-use)
- **Business Logic Bypass**: Adding negative quantities to cart to reduce price
- **Insecure Workflow**: Allowing password reset without verification

**Race Condition Test**:

```python
# VULNERABLE: Race condition in banking
def transfer_funds(from_account, to_account, amount):
    balance = get_balance(from_account)
    if balance >= amount:
        # Race condition: Another request might debit here
        deduct(from_account, amount)
        credit(to_account, amount)
        return True
    return False

# EXPLOIT: Double-spend attack
# Send two simultaneous transfer requests with same balance
# Both checks pass before either deduction completes

# SECURE: Database-level atomicity
def transfer_secure(from_account, to_account, amount):
    with db.transaction():
        # Lock row for update (pessimistic locking)
        result = db.execute("""
            UPDATE accounts 
            SET balance = balance - %s 
            WHERE id = %s AND balance >= %s
            RETURNING balance
        """, (amount, from_account, amount))
        
        if result.rowcount == 0:
            raise InsufficientFunds()
        
        db.execute("UPDATE accounts SET balance = balance + %s WHERE id = %s",
                  (amount, to_account))
```

### **37.2.5 A05: Security Misconfiguration**

**Description**: Security settings not securely defined, maintained, or implemented. This is the most commonly seen vulnerability.

**Common Issues**:
- Default credentials (`admin`/`admin`)
- Unnecessary features enabled (unused ports, services)
- Verbose error messages exposing stack traces
- Missing security headers (CSP, HSTS, X-Frame-Options)
- Directory listing enabled

**Security Headers Testing**:

```python
# Flask example of security headers
from flask import Flask
from flask_talisman import Talisman

app = Flask(__name__)

# Security headers middleware
Talisman(app,
    force_https=True,
    strict_transport_security=True,
    content_security_policy={
        'default-src': "'self'",
        'script-src': "'self'",
        'style-src': "'self' 'unsafe-inline'"
    },
    referrer_policy='strict-origin-when-cross-origin',
    feature_policy={
        'geolocation': "'none'",
        'microphone': "'none'",
        'camera': "'none'"
    }
)

# Manual header verification
@app.after_request
def set_security_headers(response):
    response.headers['X-Content-Type-Options'] = 'nosniff'
    response.headers['X-Frame-Options'] = 'DENY'
    response.headers['X-XSS-Protection'] = '1; mode=block'
    response.headers['Permissions-Policy'] = 'accelerometer=(), camera=(), geolocation=(), gyroscope=(), magnetometer=(), microphone=(), payment=(), usb=()'
    return response
```

**Test Cases**:
- Verify `X-Frame-Options` prevents clickjacking
- Check `Content-Security-Policy` blocks XSS
- Ensure `Strict-Transport-Security` enforces HTTPS
- Verify no `Server` header exposes version info

### **37.2.6 A06: Vulnerable and Outdated Components**

**Description**: Using components (libraries, frameworks, modules) with known vulnerabilities.

**Testing Approach**:
- **Software Composition Analysis (SCA)**: Identify all dependencies and their versions
- **Vulnerability Scanning**: Check against CVE databases (NIST NVD)
- **Transitive Dependencies**: Vulnerabilities in dependencies of dependencies

```bash
# Python: Check dependencies for vulnerabilities
pip install safety
safety check -r requirements.txt

# Node.js
npm audit
# or
yarn audit

# Java
mvn dependency-check:check

# OWASP Dependency Check CLI
dependency-check.sh --project "My App" --scan ./src --format HTML
```

**Remediation**:
- Maintain Software Bill of Materials (SBOM)
- Automate vulnerability scanning in CI/CD
- Pin specific versions, not ranges
- Regular update cycles (patch management)

### **37.2.7 A07: Identification and Authentication Failures**

**Description**: Previously "Broken Authentication". Weaknesses allowing attackers to compromise passwords, keys, session tokens.

**Vulnerabilities**:
- Brute force attacks (no rate limiting)
- Weak password policies
- Session fixation
- Credential stuffing
- Multi-factor authentication bypass

**Authentication Testing**:

```python
# VULNERABLE: No rate limiting
@app.route('/login', methods=['POST'])
def login():
    username = request.form['username']
    password = request.form['password']
    user = User.query.filter_by(username=username).first()
    if user and user.check_password(password):
        session['user_id'] = user.id
        return redirect('/dashboard')
    return 'Invalid credentials', 401

# SECURE: Rate limiting + account lockout
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

limiter = Limiter(app, key_func=get_remote_address)

@app.route('/login', methods=['POST'])
@limiter.limit("5 per minute")  # Rate limiting
def login_secure():
    username = request.form['username']
    password = request.form['password']
    
    # Check for account lockout
    if is_account_locked(username):
        return 'Account temporarily locked', 403
    
    user = User.query.filter_by(username=username).first()
    
    if not user or not user.check_password(password):
        record_failed_attempt(username)
        if get_failed_attempts(username) >= 5:
            lock_account(username, duration=300)  # 5 minutes
        return 'Invalid credentials', 401
    
    # Successful login
    clear_failed_attempts(username)
    regenerate_session()  # Prevent session fixation
    session['user_id'] = user.id
    return redirect('/dashboard')

def regenerate_session():
    """Prevent session fixation attacks"""
    old_data = dict(session)
    session.clear()
    session.update(old_data)
    session['_fresh'] = True
```

### **37.2.8 A08: Software and Data Integrity Failures**

**Description**: Code and infrastructure that does not protect against integrity violations. Includes insecure deserialization and supply chain attacks.

**Insecure Deserialization Example**:

```python
# VULNERABLE: Pickle deserialization
import pickle
import base64

@app.route('/load_session')
def load_session():
    data = request.cookies.get('session')
    obj = pickle.loads(base64.b64decode(data))  # DANGEROUS!
    return obj.username

# ATTACK: Craft malicious pickle payload
# pickle can execute arbitrary code during unpickling

# SECURE: Use JSON with schema validation
import json
from marshmallow import Schema, fields, ValidationError

class SessionSchema(Schema):
    user_id = fields.Integer(required=True)
    role = fields.String(validate=lambda x: x in ['user', 'admin'])

@app.route('/load_session')
def load_session_secure():
    try:
        data = json.loads(base64.b64decode(request.cookies.get('session')))
        schema = SessionSchema()
        result = schema.load(data)
        return result
    except (json.JSONDecodeError, ValidationError):
        return 'Invalid session', 400
```

**Supply Chain Security**:
- Verify signatures on dependencies (Sigstore, GPG)
- Use private registries with vulnerability scanning
- Pin Docker image digests, not tags

### **37.2.9 A09: Security Logging and Monitoring Failures**

**Description**: Insufficient logging, detection, and response to security events.

**Requirements**:
- Log all authentication attempts (success and failure)
- Log access control failures
- Log input validation failures
- Ensure logs are tamper-proof (write-once storage)
- Include correlation IDs for request tracing

**Secure Logging**:

```python
import logging
import hashlib
from flask import request, g

# Structured logging for security events
security_logger = logging.getLogger('security')

def log_security_event(event_type, severity, details):
    """Standardized security logging"""
    log_entry = {
        'timestamp': datetime.utcnow().isoformat(),
        'event_type': event_type,
        'severity': severity,
        'source_ip': request.remote_addr,
        'user_agent': request.headers.get('User-Agent'),
        'user_id': getattr(g, 'user_id', None),
        'session_id': hashlib.sha256(session.get('id', '').encode()).hexdigest()[:16],  # Hashed
        'endpoint': request.endpoint,
        'method': request.method,
        'details': details
    }
    
    # Log to SIEM-compatible format (JSON)
    security_logger.info(json.dumps(log_entry))

# Usage
@app.route('/login', methods=['POST'])
def login():
    username = request.form['username']
    user = authenticate(username, request.form['password'])
    
    if user:
        log_security_event('AUTH_SUCCESS', 'INFO', {'username': username})
        return redirect('/dashboard')
    else:
        log_security_event('AUTH_FAILURE', 'WARNING', {
            'username': username,
            'reason': 'invalid_credentials'
        })
        return 'Invalid credentials', 401
```

### **37.2.10 A10: Server-Side Request Forgery (SSRF)**

**Description**: The application fetches remote resources without validating the user-supplied URL, allowing attackers to force the server to make requests to internal services.

**Vulnerable Code**:

```python
# VULNERABLE
@app.route('/fetch_url')
def fetch_url():
    url = request.args.get('url')
    response = requests.get(url)  # No validation!
    return response.text

# ATTACK: ?url=http://localhost:8080/admin
# Server fetches internal admin panel
# Or: ?url=file:///etc/passwd
# Reads local files

# SECURE: URL validation
from urllib.parse import urlparse
import ipaddress

def is_safe_url(url):
    parsed = urlparse(url)
    
    # Block non-HTTP schemes
    if parsed.scheme not in ['http', 'https']:
        return False
    
    # Resolve hostname
    try:
        hostname = parsed.hostname
        if not hostname:
            return False
            
        # Check for private IPs
        ip = ipaddress.ip_address(hostname)
        if ip.is_private or ip.is_loopback or ip.is_reserved:
            return False
    except ValueError:
        # It's a domain name, resolve it
        import socket
        try:
            resolved = socket.gethostbyname(hostname)
            ip = ipaddress.ip_address(resolved)
            if ip.is_private or ip.is_loopback:
                return False
        except socket.gaierror:
            return False
    
    # Whitelist approach (best)
    allowed_domains = ['api.trusted-service.com', 'cdn.example.com']
    if hostname not in allowed_domains:
        return False
        
    return True

@app.route('/fetch_url')
def fetch_url_secure():
    url = request.args.get('url')
    if not is_safe_url(url):
        return 'Invalid URL', 400
    
    # Additional: Use dedicated network segment with no internal access
    response = requests.get(url, timeout=5)
    return response.text
```

---

## **37.3 Threat Modeling**

Threat modeling is the process of identifying potential threats and defining countermeasures to prevent or mitigate them.

### **37.3.1 STRIDE Methodology**

STRIDE identifies six categories of threats:

| Category | Description | Example |
|----------|-------------|---------|
| **S**poofing | Impersonating someone/something | Stolen session tokens |
| **T**ampering | Modifying data/code | SQL injection, parameter tampering |
| **R**epudiation | Denying an action | Missing audit logs |
| **I**nformation Disclosure | Exposing information | Verbose errors, IDOR |
| **D**enial of Service | Disrupting availability | Resource exhaustion |
| **E**levation of Privilege | Gaining unauthorized permissions | Vertical privilege escalation |

### **37.3.2 Attack Trees**

Visual representation of attack paths:

```
Goal: Access Admin Panel
├── Path 1: Authentication Bypass
│   ├── 1.1 SQL Injection in login
│   ├── 1.2 Brute force weak password
│   └── 1.3 Session fixation
├── Path 2: Privilege Escalation
│   ├── 2.1 Modify role parameter in JWT
│   └── 2.2 Mass assignment vulnerability
└── Path 3: Infrastructure
    ├── 3.1 SSRF to internal admin
    └── 3.2 Default credentials on admin panel
```

### **37.3.3 Threat Modeling Process**

1. **Identify Assets**: What are you protecting? (User data, API keys, intellectual property)
2. **Create Architecture Diagrams**: Data flow diagrams (DFD) showing trust boundaries
3. **Identify Threats**: Apply STRIDE to each component
4. **Rate Threats**: Risk = Likelihood × Impact
5. **Mitigate**: Implement controls or accept risk
6. **Validate**: Verify controls work through testing

---

## **37.4 Security Testing Types**

### **37.4.1 Static Application Security Testing (SAST)**

Analyzes source code without executing it.

**Tools**:
- **SonarQube**: Multi-language security and quality analysis
- **Checkmarx**: Enterprise SAST
- **Semgrep**: Lightweight, fast pattern matching
- **Bandit**: Python-specific security linter

```bash
# Bandit example
bandit -r ./src -f html -o report.html

# Semgrep
semgrep --config=auto .
```

### **37.4.2 Dynamic Application Security Testing (DAST)**

Tests running applications by simulating attacks.

**Tools**:
- **OWASP ZAP**: Open-source web vulnerability scanner
- **Burp Suite**: Professional web security testing platform
- **Nikto**: Web server scanner

```bash
# ZAP baseline scan (CI/CD friendly)
docker run -t owasp/zap2docker-stable zap-baseline.py \
  -t https://target.example.com \
  -g gen.conf \
  -r report.html
```

### **37.4.3 Interactive Application Security Testing (IAST)**

Combines SAST and DAST by instrumenting the application during runtime.

**Tools**:
- **Contrast Security**: Runtime application self-protection
- **Seeker**: Synopsys IAST solution
- **Checkmarx IAST**: Runtime analysis

**Advantage**: Detects vulnerabilities with runtime context (actual data flow, not theoretical).

### **37.4.4 Software Composition Analysis (SCA)**

Identifies open-source components and their vulnerabilities.

**Tools**:
- **Snyk**: Dependency vulnerability scanning
- **WhiteSource (Mend)**: License and security compliance
- **FOSSA**: Open-source license compliance

---

## **37.5 Authentication and Authorization Testing**

### **37.5.1 Session Management Testing**

**Test Cases**:
- Session timeout enforcement
- Session fixation prevention
- Concurrent session handling
- Secure session ID generation (entropy)
- Session invalidation on logout

```python
# Test session entropy
import secrets

def test_session_entropy():
    """Verify session IDs are cryptographically random"""
    sessions = [secrets.token_urlsafe(32) for _ in range(1000)]
    
    # Check uniqueness
    assert len(set(sessions)) == 1000, "Duplicate session IDs detected!"
    
    # Check length (minimum 128 bits entropy)
    for s in sessions:
        decoded = base64.urlsafe_b64decode(s + '==')
        assert len(decoded) >= 16, "Insufficient session entropy"
```

### **37.5.2 JWT Security Testing**

JSON Web Tokens are commonly used but easily misconfigured.

**Vulnerabilities to Test**:
- **None Algorithm**: `alg: "none"` bypasses signature
- **Weak Secrets**: Brute force HS256 secrets
- **Key Confusion**: RS256 public key used as HS256 secret
- **Expired Tokens**: Clock skew issues

```python
import jwt
from jwt.exceptions import InvalidTokenError

def test_jwt_security(token, secret_or_key):
    """Test JWT implementation"""
    issues = []
    
    # Test 1: Verify algorithm is not 'none'
    header = jwt.get_unverified_header(token)
    if header.get('alg') == 'none':
        issues.append("CRITICAL: Algorithm 'none' accepted")
    
    # Test 2: Verify expiration is checked
    try:
        jwt.decode(token, secret_or_key, algorithms=['HS256'], options={"verify_exp": True})
    except jwt.ExpiredSignatureError:
        pass  # Expected for expired tokens
    except InvalidTokenError:
        issues.append("Token validation failed")
    
    # Test 3: Check for weak secrets (if testing environment)
    # Attempt common weak secrets
    weak_secrets = ['secret', 'password', '123456', 'admin']
    for weak in weak_secrets:
        try:
            jwt.decode(token, weak, algorithms=['HS256'])
            issues.append(f"Weak secret detected: {weak}")
            break
        except:
            pass
    
    return issues
```

---

## **37.6 Input Validation and Fuzzing**

### **37.6.1 Boundary Testing**

Test limits of input fields:

```python
def test_input_boundaries():
    """Test boundary conditions"""
    test_cases = [
        ("Empty string", ""),
        ("Max length + 1", "A" * 1001),  # If max is 1000
        ("Null byte", "user\x00admin"),   # Null byte injection
        ("Unicode", "用户"),              # Internationalization
        ("Special chars", "'; DROP TABLE users; --"),
        ("Newlines", "user\nadmin"),
        ("Whitespace", "   user   "),      # Trim handling
    ]
    
    for name, input_val in test_cases:
        response = submit_form(input_val)
        assert response.status_code != 500, f"Server error on {name}"
```

### **37.6.2 Fuzzing**

Automated generation of malformed inputs to trigger crashes.

```python
# Simple fuzzer example
import random
import string

def fuzz_input(base_input, iterations=100):
    """Mutate base input to find crashes"""
    for i in range(iterations):
        mutated = list(base_input)
        
        # Random operations
        op = random.choice(['insert', 'delete', 'replace', 'repeat'])
        
        if op == 'insert':
            pos = random.randint(0, len(mutated))
            char = random.choice(string.printable)
            mutated.insert(pos, char)
        elif op == 'delete' and len(mutated) > 0:
            pos = random.randint(0, len(mutated) - 1)
            del mutated[pos]
        elif op == 'replace' and len(mutated) > 0:
            pos = random.randint(0, len(mutated) - 1)
            mutated[pos] = random.choice(string.printable)
        
        test_input = ''.join(mutated)
        
        try:
            response = requests.post(TARGET_URL, data={'input': test_input}, timeout=5)
            if response.status_code == 500:
                print(f"Server error with input: {repr(test_input)}")
        except Exception as e:
            print(f"Exception with input {repr(test_input)}: {e}")
```

---

## **37.7 Compliance and Standards**

### **37.7.1 PCI-DSS (Payment Card Industry)**

Requirements for applications handling credit cards:
- Requirement 6.5: Address common coding vulnerabilities (OWASP Top 10)
- Requirement 8.2: Strong authentication
- Requirement 11.3: Penetration testing

### **37.7.2 GDPR (General Data Protection Regulation)**

EU data protection requirements:
- Data minimization (only collect necessary data)
- Right to erasure (secure deletion testing)
- Encryption of personal data
- Breach notification (72 hours)

### **37.7.3 HIPAA (Health Insurance Portability and Accountability Act)**

US healthcare data protection:
- Encryption at rest and in transit
- Access controls and audit logging
- Data integrity safeguards

### **37.7.4 Security Requirements Traceability Matrix**

Map security requirements to test cases:

| Requirement | Control | Test Case | Status |
|-------------|---------|-----------|--------|
| PCI 6.5.1 | SQL Injection Prevention | TC-SQL-001: Attempt injection in login form | Pass |
| PCI 8.2.3 | Password Complexity | TC-AUTH-003: Verify min length 12 chars | Pass |
| GDPR 32 | Encryption at Rest | TC-CRYPT-001: Verify AES-256 for PII | Pass |

---

## **Chapter Summary**

### **Key Takeaways:**

**Security Principles (37.1):**
- **CIA Triad**: Confidentiality, Integrity, Availability—the foundation of all security testing
- **Defense in Depth**: No single point of failure; security at every layer
- **Least Privilege**: Minimum necessary permissions reduce blast radius
- **Fail Secure**: When security controls fail, default to deny, not allow

**OWASP Top 10 (37.2):**
- **Broken Access Control**: Test IDOR, path traversal, privilege escalation with horizontal/vertical access attempts
- **Cryptographic Failures**: Verify strong hashing (bcrypt/Argon2), TLS 1.3, no hardcoded keys
- **Injection**: Use parameterized queries; never concatenate SQL; validate all inputs
- **Insecure Design**: Race conditions, business logic flaws—test for time-of-check to time-of-use issues
- **Security Misconfiguration**: Verify security headers (CSP, HSTS), disable verbose errors, change defaults
- **Vulnerable Components**: Maintain SBOM, automate SCA scanning, patch within 30 days of CVE release
- **Auth Failures**: Implement rate limiting, account lockout, MFA, secure session management
- **Integrity Failures**: Avoid pickle/deserialization of untrusted data; verify dependency signatures
- **Logging Failures**: Log all auth events (success/failure), use structured JSON, ensure tamper-proof storage
- **SSRF**: Validate URLs against allowlists, block private IP ranges, use network segmentation

**Threat Modeling (37.3):**
- **STRIDE**: Spoofing, Tampering, Repudiation, Info Disclosure, DoS, Elevation of Privilege
- **Attack Trees**: Visualize paths to compromise; prioritize high-impact, low-effort attacks
- **Risk Rating**: Likelihood × Impact; focus testing on high-risk components

**Testing Types (37.4):**
- **SAST**: Source code analysis (SonarQube, Semgrep)—find bugs early
- **DAST**: Runtime testing (ZAP, Burp)—finds configuration issues
- **IAST**: Instrumented testing—high accuracy with runtime context
- **SCA**: Dependency scanning—critical for supply chain security

**Authentication Testing (37.5):**
- **Session Management**: Test fixation, timeout, entropy, concurrent sessions
- **JWT**: Verify algorithm != "none", strong secrets, expiration checks, no sensitive data in payload

**Input Validation (37.6):**
- **Boundary Testing**: Empty strings, max length, null bytes, Unicode
- **Fuzzing**: Automated mutation testing to find crashes and injection points

**Compliance (37.7):**
- **PCI-DSS**: Required for payment processing; annual penetration testing
- **GDPR**: Privacy by design; right to erasure; breach notification
- **Traceability**: Map requirements to test cases for audit evidence

**Security Testing Best Practices:**
1. **Shift Left**: Integrate SAST into IDE and CI/CD pipelines
2. **Defense in Depth**: Assume any single control may fail
3. **Never Trust Input**: Validate on server side, not just client side
4. **Secure by Default**: Opt-out security, not opt-in
5. **Least Privilege**: Database connections, file permissions, API scopes
6. **Audit Everything**: Log security events with correlation IDs
7. **Stay Current**: Subscribe to security advisories for dependencies

---

## **📖 Next Chapter: Chapter 38 - Security Testing Tools**

Now that you understand the fundamental vulnerabilities and testing methodologies, **Chapter 38** will provide hands-on guidance with the **industry-standard tools** used to identify and exploit security vulnerabilities.

In **Chapter 38**, you will master:

- **OWASP ZAP**: Automated vulnerability scanning, spidering, active scanning, and API security testing
- **Burp Suite**: Professional web application security testing—Intruder, Repeater, Scanner, and Sequencer
- **SQLMap**: Automated SQL injection detection and exploitation
- **Nmap**: Network discovery and vulnerability scanning
- **Metasploit**: Exploitation framework for penetration testing
- **Wireshark**: Packet analysis for network security testing
- **Hashcat/John the Ripper**: Password cracking for assessing password strength
- **Mobile Security**: MobSF (Mobile Security Framework) for Android/iOS testing
- **CI/CD Security**: Integrating security scanning into DevSecOps pipelines

This chapter will transition you from **security theory to practical exploitation**, teaching you how to use professional tools to identify vulnerabilities before attackers do.

**Continue to Chapter 38 to learn the practical tools of ethical hacking and security testing!**